In [41]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods")
using Revise
using OrbitalNeuralMethods

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods`


In [42]:
import Random
rng = Random.MersenneTwister();

In [43]:
n = 2
ω = 0.25
ham = HOCoulomb(ω, shielding = 0.25);

l = 20
basis = SpinBasis(HOBasis(l, ω))
grid = [x for x in range(-10, stop = 10, length = 2001)]
system = System(n, basis, grid, ham);

mixer = Alpha(0.5)
rhf = RHF(system, mixer);
rhf = compute_ground_state!(rhf, max_iter = 1000, tol = 1e-10, verbose=0);
energy(rhf)

1.1795769055185399

In [68]:
metro = Importance(equils = 1e4, samples = 1e6, step = 0.1);
#metro = Metropolis(equils = 1e4, samples = samples, step = 0.1);

In [60]:
nn = NeuralNetwork([Dense(10), Tanh, Dense(1), Exp], n=n, rng = rng);
slater = Slater(rhf);
wf = SlaterNN(slater, nn);
optimizer = ADAM(wf, lr = 0.01, max_iter = 10, tol = 1e-6);

In [70]:
@time wf, grad_results = optimize(wf, ham, metro, optimizer, nthreads = 12, verbose = true);

E = 0.822989 iter = 10/10                                      
No convergence reached, final norm of gradient was 0.011167471355192856
  4.939324 seconds (27.25 k allocations: 6.771 MiB)


In [ ]:
0.82485
0.8247

In [80]:
samples_b = 2^20
metro_b = Importance(equils = 1e6, samples = samples_b, step = 0.1);
@time result = blocking(wf, ham, metro_b, nthreads = 12);
digits = Integer(-floor(log10(result.E_err))) + 1
print("$(round(result.E, digits=digits)) ± $(round(result.E_err, digits=digits))  in $(samples_b) samples")

  2.951424 seconds (2.26 k allocations: 88.626 MiB)
0.8254 ± 0.001  in 1048576 samples

In [ ]:
#nn = NeuralNetwork([Dense(4), Tanh, Dense(1), Exp], n=n, rng = rng);
#0.8257616006071262